In [63]:
import os, sys
import pyspark

myConf=pyspark.SparkConf()
spark = pyspark.sql.SparkSession\
    .builder\
    .master("local")\
    .appName("myApp")\
    .config(conf=myConf)\
    .getOrCreate()

In [64]:
%%writefile data/ds_spark_song_yesterday.txt
Yesterday 어제는
All my troubles seemed to far away 내 모든 문제들이 참 멀리 있는 것 같았는데
Now it looks as though they're here to stay 이제는 그 모든 문제들이 여기에 있는 것처럼 보여
Oh, I believe in yesterday 오오, 어제가 좋았는데
Suddenly 갑자기 그-림-자-가 2021 그-림-자-가 2021
I'm not half the man I used to be 난 예전 내 모습의 반도 못한 사람이 되어버렸어
There's a shadow hanging over me 내 위에 그림자가 드리워져 있네
Oh!, yesterday came suddenly 오오!, 이제는 너무 갑자기 와버렸어..

Overwriting data/ds_spark_song_yesterday.txt


# 1) 단어빈도 계산



- 대소문자 구별 (Yesterday, yesterday는 동일한 단어로 인식한다)

- 불필요한 느낌표 '!', 글자 사이 하이픈 '-', 쉼표 ',', 점 '.' 등 제거

(예: oh! -> oh, '그-림-자-가' -> '그림자가' 된다)

- 한 글자 단어 제거 ('i', '내' 등)

- 숫자 제거



# 2) 빈도순으로 출력, 전체단어 수 출력



```

[(3, 'yesterday'),

 (3, 'to'),

 (3, '그림자가'),

 (2, '모든'),

 (2, '문제들이'),

 (2, '있는'),

 (2, '이제는'),

 (2, 'oh'),

 (2, '오오'),

 (2, 'suddenly'),

 (2, '갑자기')]

```

In [65]:
rdd = spark.sparkContext\
    .textFile(os.path.join("data","ds_spark_song_yesterday.txt"))

rdd.collect()

['Yesterday 어제는',
 'All my troubles seemed to far away 내 모든 문제들이 참 멀리 있는 것 같았는데',
 "Now it looks as though they're here to stay 이제는 그 모든 문제들이 여기에 있는 것처럼 보여",
 'Oh, I believe in yesterday 오오, 어제가 좋았는데',
 'Suddenly 갑자기 그-림-자-가 2021 그-림-자-가 2021',
 "I'm not half the man I used to be 난 예전 내 모습의 반도 못한 사람이 되어버렸어",
 "There's a shadow hanging over me 내 위에 그림자가 드리워져 있네",
 'Oh!, yesterday came suddenly 오오!, 이제는 너무 갑자기 와버렸어..']

In [66]:
bans = ['!','-','.',',','0','1','2','3','4','5','6','7','8','9']

def replacaBan(x):
    for ban in bans:
        x = x.replace(ban, '')
    return x

# 대소문자 구별 (Yesterday, yesterday는 동일한 단어로 인식한다)
# 불필요한 느낌표 '!', 글자 사이 하이픈 '-', 쉼표 ',', 점 '.' 등 제거
# 숫자 제거

rdd2 = rdd\
    .flatMap(lambda x: x.split())\
    .map(lambda x: x.lower())\
    .map(lambda x: replacaBan(x))\
    .map(lambda x: (x, 1))\
    .reduceByKey(lambda x,y:x+y)

rdd2.collect()


[('yesterday', 3),
 ('어제는', 1),
 ('all', 1),
 ('my', 1),
 ('troubles', 1),
 ('seemed', 1),
 ('to', 3),
 ('far', 1),
 ('away', 1),
 ('내', 3),
 ('모든', 2),
 ('문제들이', 2),
 ('참', 1),
 ('멀리', 1),
 ('있는', 2),
 ('것', 1),
 ('같았는데', 1),
 ('now', 1),
 ('it', 1),
 ('looks', 1),
 ('as', 1),
 ('though', 1),
 ("they're", 1),
 ('here', 1),
 ('stay', 1),
 ('이제는', 2),
 ('그', 1),
 ('여기에', 1),
 ('것처럼', 1),
 ('보여', 1),
 ('oh', 2),
 ('i', 2),
 ('believe', 1),
 ('in', 1),
 ('오오', 2),
 ('어제가', 1),
 ('좋았는데', 1),
 ('suddenly', 2),
 ('갑자기', 2),
 ('그림자가', 3),
 ('', 2),
 ("i'm", 1),
 ('not', 1),
 ('half', 1),
 ('the', 1),
 ('man', 1),
 ('used', 1),
 ('be', 1),
 ('난', 1),
 ('예전', 1),
 ('모습의', 1),
 ('반도', 1),
 ('못한', 1),
 ('사람이', 1),
 ('되어버렸어', 1),
 ("there's", 1),
 ('a', 1),
 ('shadow', 1),
 ('hanging', 1),
 ('over', 1),
 ('me', 1),
 ('위에', 1),
 ('드리워져', 1),
 ('있네', 1),
 ('came', 1),
 ('너무', 1),
 ('와버렸어', 1)]

In [67]:
# 공백 제거, 1글자 제거, 출현빈도 1 이하 제거

rdd3 = rdd2.filter(lambda x: x[1] > 1)\
    .filter(lambda x: x[0] != '')\
    .filter(lambda x: len(x[0]) > 1)

rdd3.collect()

[('yesterday', 3),
 ('to', 3),
 ('모든', 2),
 ('문제들이', 2),
 ('있는', 2),
 ('이제는', 2),
 ('oh', 2),
 ('오오', 2),
 ('suddenly', 2),
 ('갑자기', 2),
 ('그림자가', 3)]

In [68]:
# 빈도를 기준으로 정렬

rdd4 = rdd3.map(lambda x: (x[1],x[0]))\
    .sortByKey(False)
    
rdd4

PythonRDD[65] at RDD at PythonRDD.scala:53

In [69]:
rdd4.collect()

[(3, 'yesterday'),
 (3, 'to'),
 (3, '그림자가'),
 (2, '모든'),
 (2, '문제들이'),
 (2, '있는'),
 (2, '이제는'),
 (2, 'oh'),
 (2, '오오'),
 (2, 'suddenly'),
 (2, '갑자기')]